<a href="https://colab.research.google.com/github/yuruojie777/NLP/blob/main/UNIKEY_COMP5046_Ass1_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2022 COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing

## 1.0. Data Collection [DO NOT MODIFY THIS]

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '16g474hdNsaNx0_SnoKuqj2BuwSEGdnbt'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.csv')  

id = '1-7hj0sF3Rc5G6POKdkpbDXm_Q6BWFDPU'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.csv')  

import pandas as pd
training_data = pd.read_csv("/content/training_data.csv")
testing_data = pd.read_csv("/content/testing_data.csv")

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data)))
print("Size of testing dataset: {0}".format(len(testing_data)))
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data.iloc[-1,0], training_data.iloc[-1,1]))
print("------------------------------------")


------------------------------------
Size of training dataset: 7808
Size of testing dataset: 867
------------------------------------
------------------------------------
Sample Data
LABEL: F / SENTENCE: 'Half of it is going straight to charity, another quarter going straight to scientific research, an eighth to the parkour community, a sixteenth to towards spreading information about health and...|||Find a path or suffer more.|||http://personalitycafe.com/enneagram-personality-theory-forum/85323-enneagram-type-mbti-type-compared-statistics.html yep.|||I kind of anchor on Fi and Ne makes having Ni really fun. INFP for me as they tire me out less and our views tend to align more.|||The two ESTPs I have gotten the chance to know seem to experience much more than other people who have been on the planet for the same amount of time and are quite the renaissance (wo)men.  Is this...|||I don't really have a best friend ISTP(passion-amateur group co-founder), INTJ(intellectual and various sma

In [2]:
# Preview of the data in the csv file, which has two columns: 
# (1)type - label of the post (2)posts - the corresponding post content
training_data.head()

,type,posts
0,F,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,T,'I'm finding the lack of me in these posts ver...
2,T,'Good one _____ https://www.youtube.com/wat...
3,T,"'Dear INTP, I enjoyed our conversation the o..."
4,T,'You're fired.|||That's another silly misconce...


In [3]:
# Extract the labels and posts and store into List

# Get the list of training data (posts)
training_posts=training_data['posts'].tolist()
# Get the list of corresponding labels for the training data (posts)
training_labels=training_data['type'].tolist()

# Get the list of testing data (posts)
testing_posts=testing_data['posts'].tolist()
# Get the list of corresponding labels for the testing data (posts)
testing_labels=testing_data['type'].tolist()

## 1.1. URL Removal
*related to the section 4.2*

In [4]:
# Please comment your code

# Remove URL
def remove_url(data):
  import re
  # Url pattern
  URL_Pattern = 'http\S+'
  return [re.sub(URL_Pattern, '', sentence) for sentence in data]

## 1.2. Preprocess data (e.g. Stop words, Stemming)
*related to the section 4.2*

In [5]:
# Map contraction to full words
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
            "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
            "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
            "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
            "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
            "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
            "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
            "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
            "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
            "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
            "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
            "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
            "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
            "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
            "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
            "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
            "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
            "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
            "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
            "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
            "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
            "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def remove_contraction(data):
  result = []
  for x in data:  
    words = x[1:-1].split(' ')
    link = ' '
    result_words = []
    for word in words:
      if word in contraction_dict.keys():
        result_words.append(contraction_dict[word])
      else: result_words.append(word)
    result.append(link.join(result_words))
  return result


In [6]:
# Remove puncts
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def remove_punctuation(data):
  result = []
  for sentence in data:
    sentence = str(sentence)
    for punct in puncts:
      if punct in sentence:
        sentence = sentence.replace(punct, '')
    result.append(sentence)
  return result


In [7]:
# Tokenization
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize


def tokenize_data(data):
  return [word_tokenize(sentence) for sentence in data]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# 1. Remove stop words

stop_words = sw.words()
def remove_stop_words(data):
  return [[w for w in sentence if not w in stop_words] for sentence in data]


In [14]:
# 2. Stemming
from nltk.stem.snowball import *
stemmer = SnowballStemmer('english')

def stemming(data):
  return [[stemmer.stem(word) for word in sentence] for sentence in data]

In [9]:
# 3. Lemmatisation

nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatisation(data):
  return [[lemmatizer.lemmatize(word) for word in sentence] for sentence in data]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# 4. Remove numbers
def remove_numbers(data):
  result = []
  for sentence in data:
    temp = []
    for word in sentence:
      word = re.sub(r'\d+', '', word)
      if len(word) != 0:
        temp.append(word)
    result.append(temp)
  return result

In [11]:
# posts without URL

training_data_after_removing_url_punct_contraction = remove_punctuation(remove_contraction(remove_url(training_posts)))
testing_data_after_removing_url_punct_contraction = remove_punctuation(remove_contraction(remove_url(testing_posts)))
for i in range(0,3):
  print(training_data_after_removing_url_punct_contraction[i])
  print(testing_data_after_removing_url_punct_contraction[i])


 and intj moments    sportscenter not top ten plays    pranksWhat has been the most lifechanging experience in your life     On repeat for most of todayMay the PerC Experience immerse youThe last thing my INFJ friend posted on his facebook before committing suicide the next day Rest in peace    ENFJ7 Sorry to hear of your distress Its only natural for a relationship to not be perfection all the time in every moment of existence Try to figure the hard times as times of growth as84389  84390     Welcome and stuff  Game Set MatchProzac wellbrutin at least thirty minutes of moving your legs and I do not mean moving them while sitting in your same desk chair weed in moderation maybe try edibles as a healthier alternativeBasically come up with three items you have determined that each type or whichever types you want to do would more than likely use given each types cognitive functions and whatnot when left byAll things in moderation  Sims is indeed a video game and a good one at that Note a

In [12]:
# Combination of  1&2
training_token = tokenize_data(training_data_after_removing_url_punct_contraction)
testing_token = tokenize_data(testing_data_after_removing_url_punct_contraction)
for i in range(0,3):
  print(training_token[i])
  print(testing_token[i])

['and', 'intj', 'moments', 'sportscenter', 'not', 'top', 'ten', 'plays', 'pranksWhat', 'has', 'been', 'the', 'most', 'lifechanging', 'experience', 'in', 'your', 'life', 'On', 'repeat', 'for', 'most', 'of', 'todayMay', 'the', 'PerC', 'Experience', 'immerse', 'youThe', 'last', 'thing', 'my', 'INFJ', 'friend', 'posted', 'on', 'his', 'facebook', 'before', 'committing', 'suicide', 'the', 'next', 'day', 'Rest', 'in', 'peace', 'ENFJ7', 'Sorry', 'to', 'hear', 'of', 'your', 'distress', 'Its', 'only', 'natural', 'for', 'a', 'relationship', 'to', 'not', 'be', 'perfection', 'all', 'the', 'time', 'in', 'every', 'moment', 'of', 'existence', 'Try', 'to', 'figure', 'the', 'hard', 'times', 'as', 'times', 'of', 'growth', 'as84389', '84390', 'Welcome', 'and', 'stuff', 'Game', 'Set', 'MatchProzac', 'wellbrutin', 'at', 'least', 'thirty', 'minutes', 'of', 'moving', 'your', 'legs', 'and', 'I', 'do', 'not', 'mean', 'moving', 'them', 'while', 'sitting', 'in', 'your', 'same', 'desk', 'chair', 'weed', 'in', 'mod

In [15]:
after_stemming_training = stemming(training_token)
after_stemming_testing = stemming(testing_token)
for i in range(0,3):
  print(after_stemming_training[i])
  print(after_stemming_testing[i])

['and', 'intj', 'moment', 'sportscent', 'not', 'top', 'ten', 'play', 'prankswhat', 'has', 'been', 'the', 'most', 'lifechang', 'experi', 'in', 'your', 'life', 'on', 'repeat', 'for', 'most', 'of', 'todaymay', 'the', 'perc', 'experi', 'immers', 'youth', 'last', 'thing', 'my', 'infj', 'friend', 'post', 'on', 'his', 'facebook', 'befor', 'commit', 'suicid', 'the', 'next', 'day', 'rest', 'in', 'peac', 'enfj7', 'sorri', 'to', 'hear', 'of', 'your', 'distress', 'it', 'onli', 'natur', 'for', 'a', 'relationship', 'to', 'not', 'be', 'perfect', 'all', 'the', 'time', 'in', 'everi', 'moment', 'of', 'exist', 'tri', 'to', 'figur', 'the', 'hard', 'time', 'as', 'time', 'of', 'growth', 'as84389', '84390', 'welcom', 'and', 'stuff', 'game', 'set', 'matchprozac', 'wellbrutin', 'at', 'least', 'thirti', 'minut', 'of', 'move', 'your', 'leg', 'and', 'i', 'do', 'not', 'mean', 'move', 'them', 'while', 'sit', 'in', 'your', 'same', 'desk', 'chair', 'weed', 'in', 'moder', 'mayb', 'tri', 'edibl', 'as', 'a', 'healthier'

In [16]:
after_stop_words_training = remove_stop_words(after_stemming_training)
after_stop_words_testing = remove_stop_words(after_stemming_testing)
for i in range(0,3):
  print(after_stop_words_training[i])
  print(after_stop_words_testing[i])

['intj', 'moment', 'sportscent', 'top', 'ten', 'play', 'prankswhat', 'lifechang', 'experi', 'life', 'repeat', 'todaymay', 'perc', 'experi', 'immers', 'youth', 'last', 'thing', 'infj', 'friend', 'post', 'facebook', 'befor', 'commit', 'suicid', 'next', 'day', 'rest', 'peac', 'enfj7', 'sorri', 'hear', 'distress', 'onli', 'natur', 'relationship', 'perfect', 'time', 'everi', 'moment', 'exist', 'figur', 'hard', 'time', 'time', 'growth', 'as84389', '84390', 'welcom', 'stuff', 'game', 'set', 'matchprozac', 'wellbrutin', 'least', 'thirti', 'move', 'leg', 'mean', 'move', 'desk', 'chair', 'weed', 'moder', 'mayb', 'edibl', 'healthier', 'alternativebas', 'three', 'item', 'determin', 'type', 'whichev', 'type', 'would', 'like', 'use', 'given', 'type', 'cognit', 'function', 'whatnot', 'left', 'byall', 'thing', 'moder', 'sim', 'inde', 'video', 'game', 'good', 'note', 'good', 'somewhat', 'subject', 'complet', 'promot', 'death', 'ani', 'given', 'simdear', 'enfp', 'favorit', 'video', 'game', 'grow', 'curr

In [17]:
after_removing_numbers_training = remove_numbers(after_stop_words_training)
after_removing_numbers_testing = remove_numbers(after_stop_words_testing)
for i in range(0,3):
  print(after_removing_numbers_training[i])
  print(after_removing_numbers_testing[i])

['intj', 'moment', 'sportscent', 'top', 'ten', 'play', 'prankswhat', 'lifechang', 'experi', 'life', 'repeat', 'todaymay', 'perc', 'experi', 'immers', 'youth', 'last', 'thing', 'infj', 'friend', 'post', 'facebook', 'befor', 'commit', 'suicid', 'next', 'day', 'rest', 'peac', 'enfj', 'sorri', 'hear', 'distress', 'onli', 'natur', 'relationship', 'perfect', 'time', 'everi', 'moment', 'exist', 'figur', 'hard', 'time', 'time', 'growth', 'as', 'welcom', 'stuff', 'game', 'set', 'matchprozac', 'wellbrutin', 'least', 'thirti', 'move', 'leg', 'mean', 'move', 'desk', 'chair', 'weed', 'moder', 'mayb', 'edibl', 'healthier', 'alternativebas', 'three', 'item', 'determin', 'type', 'whichev', 'type', 'would', 'like', 'use', 'given', 'type', 'cognit', 'function', 'whatnot', 'left', 'byall', 'thing', 'moder', 'sim', 'inde', 'video', 'game', 'good', 'note', 'good', 'somewhat', 'subject', 'complet', 'promot', 'death', 'ani', 'given', 'simdear', 'enfp', 'favorit', 'video', 'game', 'grow', 'current', 'favorit'



---



# 2 - Input Representation


## 2.1. Word Embedding Construction
*related to the section 4.1 and 4.3*


In [33]:
# Word2Vec-CBOW
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences=after_stop_words_training + after_stop_words_testing, size=100, window=5, min_count=5, workers=2, sg=0)

In [45]:
import pprint
similar_words= wv_cbow_model.wv.most_similar("logic") # topn=10 by default
pprint.pprint(similar_words)

[('ration', 0.8451854586601257),
 ('fallaci', 0.7596102356910706),
 ('deduct', 0.7455563545227051),
 ('illog', 0.7224417924880981),
 ('object', 0.6544502973556519),
 ('precis', 0.6513552665710449),
 ('method', 0.6511602401733398),
 ('factual', 0.637279748916626),
 ('concret', 0.6342222094535828),
 ('argument', 0.630585789680481)]


In [46]:
# Word2Vec-Skip-Gram

wv_sg_model = Word2Vec(sentences=after_stop_words_training + after_stop_words_testing, size=100, window=5, min_count=5, workers=4, sg=1)

In [47]:
similar_words= wv_sg_model.wv.most_similar("logic") # topn=10 by default
pprint.pprint(similar_words)

[('ration', 0.8215256929397583),
 ('induct', 0.743458092212677),
 ('deduct', 0.7365408539772034),
 ('fallaci', 0.7171690464019775),
 ('rational', 0.7016866207122803),
 ('priori', 0.7000617980957031),
 ('faulti', 0.6990525722503662),
 ('systemat', 0.6985352635383606),
 ('irrefut', 0.6899970173835754),
 ('deism', 0.6855560541152954)]


## 2.2. Pretrained Word Embedding
*related to the section 4.3*

In [23]:
# glove-twitter-100

import gensim.downloader as api
word_emb_model = api.load("glove-twitter-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


In [25]:
# Get the Embedding lookup table
import numpy as np
emb_dim = word_emb_model.vector_size
print(emb_dim)
emb_table = []
for i, word in enumerate(word_list):
    if word in word_emb_model:
        emb_table.append(word_emb_model[word])
    else:
        emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)

100


In [27]:
print(emb_table[:20])

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.67996001  0.32203999  1.54990005 ...  0.0032953   0.16005
   0.033684  ]]


In [ ]:
# glove-wiki-gigaword-100

## 2.3. Input Concatenation
*related to the section 4.3*

In [4]:
# Contacte model trained by myself and pretrained model
data1 = [[1,2,3],[4,5,6]]
data2 = [[0.1,0.2,0.3],[0.4,0.5,0.6]]
def get_input_training_data(data1, data2):
  result = []
  for i in range(0,len(data1)):
    result.append(data1[i]+data2[i])
  return result

[[1, 2, 3, 0.1, 0.2, 0.3], [4, 5, 6, 0.4, 0.5, 0.6]]


# 3 - Model Implementation

### 3.1. Build Sequence Model (Bi-directional model)
*related to the section 4.4*

In [ ]:
# Please comment your code

### 3.2. Train Sequence Model (Bi-directional model)

*related to the section 4.4*

Note that it will not be marked if you do not display the Training Loss and the Number of Epochs in the Assignment 1 ipynb.


In [ ]:
# Please comment your code

# 4 - Evaluation


## 4.1. Word Embedding Evaluation
You are to apply Semantic-Syntactic word relationship tests for the trained word embeddings and visualise the result of Semantic-Syntactic word relationship tests.
Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence and justification*)




In [ ]:
# Please comment your code

## 4.2. Performance Evaluation with Data Processing Techiques


You are required to evaluate with the testing dataset and provide the table with f1 of test set.
Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence and justification*)



In [ ]:
# Please comment your code

## 4.3. Performance Evaluation with Different Input


You are required to evaluate with the testing dataset and provide the table with f1 of test set.
Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence and justification*)

In [ ]:
# Please comment your code

## 4.4. Performance Evaluation with Different Sequence Models


You are required to evaluate with the testing dataset and provide the table with f1 of test set.
Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence and justification*)

In [ ]:
# Please comment your code

## 4.5. HyperParameter Testing
*You are required to draw a graph(y-axis: f1, x-axis: epoch) for test set and explain the optimal number of epochs based on the learning rate you have already chosen.* Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence and justification*)



In [ ]:
# Please comment your code

#5 - Test your model via Colab Form Fields User Interface

You are required to design a user interface so that user can input a textual sentence via the colab form fields user interface to get the personality type classification result from your trained model. *You can just modify based on the following Colab Form Fields template*

In [ ]:
#@title Personality Type Prediction

text = "" #@param {type:"string"}


Predicted Personality Type: T


# Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed* 

In [ ]:
# If you used OOP style, use this section